In [1]:
%run stdPackages.ipynb
os.chdir(d['py'])
from modelData import *
from gmsPython import nestingTree
from mEmissions import targetsFromSYT

No clean-up of work-folder


# Create Model Data

This goes through the same steps as ```ModelData```, but splits up households into two blocks: Ramsey and H2M consumers. In this notebook, we split sectors by assuming that everything else is identical: Same preferences, initial assets, etc. etc.. They supply identical labor.

## 0. Preliminaries

To start with, load a raw IO database from the ```IOdata``` project. We use one that is used in the National Climate Policy (NCP) project with baseline year 2019. Set other global attributes used throughout (e.g. years for time horizon, name of database) and load database:

In [2]:
t0 = 2019 # baseline year
ioName = f'IO{t0}_NCPGR' # name of database
f_IOdata = os.path.join(d['project'], 'IOdata','data','processedData', ioName) # points to relevant raw database
T = t0+100 # time horizon
name = 'NCPGR_H2M0_db'
db = GpyDB(f_IOdata, name = name, ws = d['work'])
ws = db.ws # use this as the main workspace throughout

For relevant variables, add yearly index to data (this is currently all of them). Next, add some basic subsets related to the time index to the database:

In [3]:
def add_t0(k):
    k.index = stdSort(pd.MultiIndex.from_frame(k.index.to_frame(index=False).assign(t = t0))) 
[add_t0(db(k)) for k in db.getTypes(['var','par'])]; # for all variables and parameters in the database, add the 't' dimension:
addTimeToDB(t0, T, db) # add a number of relevant time subsets 

## 1. Production module

Arrange data for the production modules. Currently, these are all nested production functions of some sort, that are initialized from a nesting tree. Here, we provide the structure of such a nesting tree. Next, we provide some basic parameter values like elasticities.

### 1.1. Nesting structure

Generally, we consider the following nesting structure:

In [4]:
mFull = pd.MultiIndex.from_tuples([('KELM', 'RxE'), ('KELM','KEL'), 
                                                    ('KEL','L'), ('KEL','KE'), 
                                                    ('KE','K'), ('KE','E'),
                                                    ('E','Energy_input'),('E','Energy_F')], names = ['n','nn'])

The materials nest excluding energy (```RxE```) is a CES nest of different final goods that compete with similar foreign ones. We use the syntax ```RxEym_x``` to denote the intermediate good that consists of ```x``` and the relevant foreign good:

In [5]:
nF_full = db('n_p')+'_F' # some of the set elements in the database may have been removed, because they are not used in a single sector
n_noEnergy = db('n_p').difference(['Energy']) 
nF_noEnergy= nF_full.difference(['Energy_F'])
RxE1 = pd.MultiIndex.from_product([['RxE'], 'RxEym_'+n_noEnergy], names = ['n','nn']) # mapping from RxE to intermediate goods
RxE2 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), n_noEnergy+'_input'], names = ['n','nn']) # mapping from intermediate good to domestic one 
RxE3 = pd.MultiIndex.from_arrays([RxE1.get_level_values('nn'), nF_noEnergy], names = ['n','nn']) # mapping from intermediate good to foreign one
mFull = reduce(pd.Index.union, [mFull, RxE1, RxE2, RxE3]) # Combine the four indices

If a sector uses inputs from all other sectors and foreign sectors, this is the mapping that describes its production structure. We allow for each sector to have a unique nesting structure by adding the sector index:

In [6]:
m = pyDatabases.cartesianProductIndex([db('s_p'), mFull])

Next, it may not be the case that all sectors rely on all types of inputs. In this case, we "trim" the nesting tree by eliminating the knots that are unused:

In [7]:
sparsity = adj.rc_pd(db('vD')[db('vD')!=0], db('s_p')).droplevel('t').index # what types of inputs do the sectors actually use in the data
# recall that inputs are called '_input' for products from the domestic sectors. Map the sparsity index to these:
mapNames = pd.Series(db('n_p')+'_input', index = db('n_p')).combine_first(
            pd.Series(sparsity.levels[-1], index = sparsity.levels[-1]))
sparsity = sparsity.set_levels(sparsity.levels[-1].map(mapNames), level = -1)

m = nestingTree.trimNestingStructure(m, sparsity)

Finally, we replace the upper-most level ```KELM``` with the name of the sector instead:

In [8]:
df = m.to_frame(index=False)
df.loc[df.n == 'KELM','n'] = df.loc[df.n == 'KELM', 's']
m = pd.MultiIndex.from_frame(df)
db['nestProduction'] = m

### 1.2. Elasticities

Define some basic elasticities for the nesting tree structure:

In [9]:
sigmaOut = pd.Series(0.5, index = m[m.get_level_values('s')==m.get_level_values('n')].droplevel('nn').unique(), name = 'sigma') 
sigmaKEL = pd.Series(0.5, index  = m[m.get_level_values('n')=='KEL'].droplevel('nn').unique(), name = 'sigma')
sigmaKE = pd.Series(0.6, index  = m[m.get_level_values('n')=='KE'].droplevel('nn').unique(), name = 'sigma')
sigmaE = pd.Series(0.5, index  = m[m.get_level_values('n')=='E'].droplevel('nn').unique(), name = 'sigma')
sigmaRxE = pd.Series(0.1, index = m[m.get_level_values('n')=='RxE'].droplevel('nn').unique(), name = 'sigma')
sigmaRxEym = pd.Series(5, index = m[m.get_level_values('n').str.contains('RxEy')].droplevel('nn').unique(), name = 'sigma')
sigma = pd.concat([sigmaOut, sigmaKEL, sigmaKE, sigmaE, sigmaRxE, sigmaRxEym], axis = 0 )
db.aom(sigma, name = 'sigma') # add or merge (aom) to the database

### 1.3. Emissions 

In this data, we only have CO2 emissions on aggregate sectoral levels. This splits up the emissions onto specific outputs based on the share of the value of the output (if a sector produces more than one output):

In [10]:
output = adj.rc_pd(db['qS'], db['s_p']) # outputs from the production sectors
inputs = adj.rc_pd(db['qD'], ('and', [db['s_p'], ('not', db['dur_p'])])) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares
db['qCO2'] = (db('qCO2') * outShares).dropna() # overwrite qCO2 to be split into potentially multiple outputs from each sector

The symbol ```vTax``` includes a category of emission taxes (based on sum of all energy taxes, a very rough measure to be honest). Here, use this to compute tax rate on emissions (not available for $t\leq 1995$ for DK data):

In [11]:
# db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)
# db['dtauCO2']= db('tauCO2').index.droplevel('t').unique()
# db['dqCO2']  = db('qCO2').index.droplevel('t').unique()
# db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / db('qCO2').groupby('t').sum()

Set taxes to zero instead:

In [12]:
# db['tauCO2'] = pd.Series(0, index = db('qCO2').index)
# db['dtauCO2']= db('tauCO2').index.droplevel('t').unique()
# db['dqCO2']  = db('qCO2').index.droplevel('t').unique()
# db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / db('qCO2').groupby('t').sum()

Set uniform taxes instead:

In [13]:
db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)
db['dqCO2']  = db('qCO2').index.droplevel('t').unique()
db['tauCO2agg'] = (db('tauCO2') * db('qCO2')).groupby('t').sum() / db('qCO2').groupby('t').sum()
db['tauCO2'] = pd.Series(0, index = db('qCO2').index)+db('tauCO2agg') # add average rate onto all emissions
db['dtauCO2']= db('tauCO2').index.droplevel('t').unique() # update dummy

C:\Users\sxj477\AppData\Local\Temp\ipykernel_7004\3862341926.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  db['tauCO2'] = adjMultiIndex.applyMult((db('vTax').xs('Emissions',level='taxTypes') / db('qCO2').replace(0,1)).fillna(0), output.index)


### 1.4. Regulation 

Specify other tax rates (on inputs, on outputs, lump sum). Here, we let the lump sum tax adjust to reflect the total tax in the baseline year. Other taxes are set to zero.

In [14]:
db.aom(adj.rc_pd(db('TotalTax'), db['s_p'])-db('vTax').xs('Emissions',level='taxTypes'), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')

Define sector-specific prices given regulation:

In [15]:
db.aom(stdSort(adj.rc_pd(((1+db('tauD'))*db('p')).dropna(), inputs)), name = 'pD')

Add the value of the "index fund" to be equal to the capital stock:

In [16]:
db['vIdxFund'] = adj.rc_pd(db('qD'), db('dur_p')).groupby('t').sum()

## 2. Investment module

The investment module is a simple production sector without durables, profits, or any such things.

### 2.1. Nesting structure

The investment module consists only of the materials nest, but includes energy:

In [17]:
dImport = adj.rc_pd(db('dImport'), db('s_i'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_i'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_i'))

*Add mappings:*

In [18]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'Materials_'+x['n'].astype(str), u = 'Materials')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [19]:
df = dImport_dom.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [20]:
df = dImport_for.to_frame(index=False).assign(u = 'Materials')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level (Materials) with the name of the sector:

In [21]:
df = m.to_frame(index=False)
df.loc[df.n == 'Materials','n'] = df.loc[df.n == 'Materials', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [22]:
db['nestInvestment'] = m

### 2.2: Elasticities:

In [23]:
sigmaI = pd.Series(.5, index = m.droplevel('nn').unique(), name = 'sigma')
db.aom(sigmaI, name = 'sigma')

### 2.3. Regulation:

The investment sector does not itself emit emissions; however, investments are still tied to emissions through this sectors' reliance on inputs from other sectors (that are emission intensive).

In [24]:
output = adj.rc_pd(db['qS'], db['s_i']) # output
inputs = adj.rc_pd(db['qD'], db['s_i']) # inputs
outShares = output/pyDatabases.pdSum(output, 'n') # output shares

There rest is added as a lump sum tax:

In [25]:
db.aom(adj.rc_pd(db('TotalTax'), db['s_i']), name = 'tauLump')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')

Define sector-specific prices given regulation:

In [26]:
db.aom(stdSort(adj.rc_pd((1+db('tauD'))*db('p'), inputs)), name = 'pD')

## 3. Households

Create mapping from old to new sector definitions:

In [27]:
sh2new = pd.MultiIndex.from_tuples([('HH', 'Ramsey'),('HH', 'H2M')], names = ['s','sAlias'])
sNotH = adj.rc_pd(db('s'), ('not', db('s_HH')))
sh2newFull = pd.MultiIndex.from_arrays([sNotH, sNotH.rename('sAlias')]).union(sh2new)
weights_hh = pd.Series([0.6, 0.4], index = pd.Index(['Ramsey','H2M'], name = 's'))
weights = pd.Series([0.6, 0.4], index = sh2new).combine_first(pd.Series(1, index = sh2newFull))

For all variables that measure quantities of values, use these weights to split up variables (for other ones, such as prices, simply repeat them):

In [28]:
aggLike = {k: {'func': 'SplitDistr', 'kwargs': {'weights': weights}} for k in ('vTax','TotalTax','vD','vS','qD','qS','tauLump')}
AggDB.aggDB(db, sh2newFull, aggLike = aggLike)

The ```aggDB``` method messes a bit with the sorting of the indices, so we'll fix it here:

In [29]:
[db[k].__setattr__('vals', stdSort(db(k))) for k in db.symbols];

Add shares again:

In [30]:
db['uHH'] = weights_hh

### 3.1. Nesting structure

The household consumption nesting structure is somewhat similar to the investment sectors, with the exception that the top nest is a consumption aggregate that captures intertemporal preferences for consumption smoothing.

In [31]:
dImport = adj.rc_pd(db('dImport'), db('s_HH'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_HH'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_HH'))

*Add mappings:*

In [32]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'C_'+x['n'].astype(str), u = 'C')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [33]:
df = dImport_dom.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [34]:
df = dImport_for.to_frame(index=False).assign(u = 'C')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [35]:
df = m.to_frame(index=False)
df.loc[df.n == 'C','n'] = 'C_'+df.loc[df.n == 'C', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [36]:
db['nestHH'] = m

Add mapping from consumption aggregate to labor:

In [37]:
db['L2C'] = pd.MultiIndex.from_arrays([db('s_HH'), pd.Index(['L']*len(db('s_HH')), name = 'n'), ('C_'+db('s_HH')).rename('nn')])

### 3.2. Elasticities/preferences:

Upper level nest:

In [38]:
sigma_HH_upper = pd.Series(0.5, index = m[m.get_level_values('n') == 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [39]:
sigma_HH_Import = pd.Series(2, index = m[m.get_level_values('n') != 'C_HH'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [40]:
sigma_HH = pd.concat([sigma_HH_upper, sigma_HH_Import], axis = 0) 
db.aom(sigma_HH, name = 'sigma')

Frisch elasticity, CRRA, time preferences:

In [41]:
db.aom(pd.Series(0.1, index = db('s_HH')), name = 'frisch')
db.aom(pd.Series(2, index = db('s_HH')), name = 'crra')
db.aom(pd.Series((1+db('g_LR'))**(db('crra'))/db('R_LR'), index = db('s_HH')), name = 'discF')

### 3.3. Regulation:

Add taxes as lump-sum:

In [42]:
output = adj.rc_pd(db('qS'), db['s_HH'])
inputs = adj.rc_pd(db('qD'), db['s_HH'])
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(pd.Series(0, index = output.index), name = 'tauS')
db.aom(adj.rc_pd(db('TotalTax'), db('s_HH')), name = 'tauLump')

*Define sector-specific prices given regulation:*

In [43]:
db.aom(stdSort(adj.rc_pd((1+db('tauD'))*db('p'), inputs)), name = 'pD')
db.aom(stdSort(adj.rc_pd(db('p')*(1-db('tauS')), output)), name = 'pS') # in this case pS is the after-tax wage rate

*Add asset value. Give all assets to Ramsey households.*

In [44]:
db.aom(pd.Series([1,0], index = pd.Index(['Ramsey','H2M'], name = 's')), name = 'uIdxFund') # share of assets in domestic shares
db.aom(pd.Series(0, index = db('s_HH')), name = 'vA_F') # value of foreign assets
db.aom(db('uIdxFund')*pd.Series(db('vIdxFund').xs(t0), index = pd.MultiIndex.from_product([db('t'), db('s_HH')])), name = 'vA')

## 4. Government

### 4.1. Nesting structure

Government consumption is nested in a similar way as household consumption - and uses the same elasticities:

In [45]:
dImport = adj.rc_pd(db('dImport'), db('s_G'))
dImport_dom = adj.rc_pd(db('dImport_dom'), db('s_G'))
dImport_for = adj.rc_pd(db('dImport_for'), db('s_G'))

*Add mappings:*

In [46]:
df = dImport.to_frame(index=False).assign(m = lambda x: 'GC_'+x['n'].astype(str), u = 'GC')
m = pd.MultiIndex.from_frame(df[['s','u','m']]).rename(['s','n','nn']) # mapping from materials to materials_x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','n']]).rename(['s','n','nn'])) # mapping from materials_x to x
m = m.union(pd.MultiIndex.from_frame(df[['s','m','nn']]).rename(['s','n','nn'])) # mapping from materials_x to x_F

*Domestic only*

In [47]:
df = dImport_dom.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Foreign only:

In [48]:
df = dImport_for.to_frame(index=False).assign(u = 'GC')
m = m.union(pd.MultiIndex.from_frame(df[['s','u','n']]).rename(['s','n','nn'])) # mapping from materials to input

Replace the upper-most level with the name of the household consumption aggregate:

In [49]:
df = m.to_frame(index=False)
df.loc[df.n == 'GC','n'] = 'GC_'+df.loc[df.n == 'GC', 's']
m = pd.MultiIndex.from_frame(df)

*Add nesting structure:*

In [50]:
db['nestG'] = m

### 4.2. Elasticities

Upper level nest:

In [51]:
sigma_G_upper = pd.Series(.9, index = m[m.get_level_values('n') == 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Lower-level (import/domestic competition):

In [52]:
sigma_G_Import = pd.Series(2, index = m[m.get_level_values('n') != 'GC_G'].droplevel('nn').unique(), name = 'sigma')

Add to database:

In [53]:
sigma_G = pd.concat([sigma_G_upper, sigma_G_Import], axis = 0) 
db.aom(sigma_G, name = 'sigma')

### 4.3. Regulation

The government regulates/taxes itself (because data says so). Because we use incomplete data on taxes (especially on household taxes/transfers), we make some adhoc adjustments here:
* Remove taxes on inventory (because the "inventory sector" is only used for completeness of the IO system).
* Set flat VAT tax rate on government consumption to target taxes from government sector.

In [54]:
inputs = adj.rc_pd(db('qD'), db('s_G'))
db['TotalTax'] = adj.rc_pd(db('TotalTax'), ('not', pd.Index(['itory'],name='s'))) # remove inventory taxes
db['d_TotalTax'] = db['TotalTax'].index.droplevel('t').unique() # what sectors pay taxes
db.aom(adj.rc_pd(db('TotalTax'), db('s_G')), name = 'tauLump', priority='first')
db.aom(pd.Series(0, index = inputs.index), name = 'tauD')
db.aom(stdSort(adj.rc_pd((1+db('tauD'))*db('p'), inputs)), name = 'pD')

## 5. Trade

Set export elasticity to 10:

In [55]:
db.aom(pd.Series(10, index = db('dExport')), name='sigma')

*Note: We should add some information on regulation on trade at some point. Here, just add flat tariff/subsidy on all exports:* 

In [56]:
avgTariffRate = adj.rc_pd(db('TotalTax'), db('s_f'))/pyDatabases.pdSum(adj.rc_pd(db('qD'), db('s_f')), 'n')
TariffRate = stdSort(adjMultiIndex.applyMult(avgTariffRate, db('dExport')))
db.aom(TariffRate, name = 'tauD')
db.aom(pd.Series(0, index = adj.rc_pd(db('TotalTax'), db('s_f')).index), name = 'tauLump')
db.aom(stdSort(adj.rc_pd(db('tauD')+db('p'), db('s_f'))), name = 'pD')

## 6. Emissions

### 6.1. Data for ```EmissionAccounts``` class

*Note: Emission intensity trends are not added here.* 

In this class, we have no abatement of emissions and no quantity targets. Regulation of CO2 is only carried out through exogenous tax rates. The required inputs for this module are computed here (the rest is already added in the production module or by the ```EmissionAccounts``` class itself):

In [57]:
db['tauDist'] = db('tauCO2')/db('tauCO2agg') # parameter used to guide CO2 regulation
# db['tauDist'] = pd.Series(1, index = db('tauCO2').index) # uniform taxes
db['tauEffCO2'] = db('tauCO2').copy() # without abatement --> identical rates.
db['uCO2'] = db('qCO2') / adj.rc_pd(db('qS'), db('dqCO2')) # emission intensity
db['qCO2agg'] = db('qCO2').groupby('t').sum() # aggregate emissions

### 6.2. Data for ```EmissionTargets``` class

The class starts from "single year targets" (SYT) formulated as specific years $t$ with targets for aggregate emissions. Based on this, the short data method ```targetsFromSYT``` returns a dictionary with various relevant subsets, but also other types of regulation (emission budgets (EB) and linear reduction paths (LRP)) that are equivalent with the SYT regulation in some way. In this example, the target is to reach net zero by 2050.

**Single year targets** (SYT):
* Define a subset of target years ```t_SYT```. Per construction, we assume that the final target is permanent (in this case the target for 2050 will be in place for 2051,..., T).
* Define a variable with target levels ```qCO2_SYT```.

A couple of additional subsets are added to allow for emission targets to be achieved with different policy rules (see the documentation notes online for more).

**Linear reduction paths** (LRP): We compute linear reduction paths between targets ```qCO2_LRP``` and set target index ```t_LRP``` to the entire time index.

**Emission Budgets** (EB):
* Based on the linear reduction paths, compute cumulative budgets for each regime (one for each original single year target).
* Let ```t_EB``` denote the end-years for each emission budget regime (coincides with original SYT). Let ```qCO2_EB``` denote the cumulative budget for the time horizon.
* Let ```t2tt_EB[t,tt]``` denote the mapping from each target year (t) and the years (tt) to sum over when reaching this target.
* If there are years after the final target, we adopt a simple SYT approach for the final years. This is done by defining a separate subset ```t_EB_SYT``` with ```qCO2_EB_SYT``` denoting the targets (they coincide with simple SYT).

In [58]:
targets1 = pd.Series([0], index = pd.Index([2050], name = 't'))
targets2 = pd.Series([0,0], index = pd.Index([2050,2060], name = 't')) # this adds an emission budget for 2050-2060 
targetSymbols1 = targetsFromSYT(targets1, db('t'), db('qCO2agg').xs(t0))
targetSymbols2 = targetsFromSYT(targets2, db('t'), db('qCO2agg').xs(t0))
[db.aom(v, name = k) for k,v in targetSymbols1.items() if k in ('qCO2_SYT','t_SYT','t_SYT_NB')];
[db.aom(v, name = k) for k,v in targetSymbols2.items() if k not in ('qCO2_SYT','t_SYT','t_SYT_NB')];

*Note:* It is important for the subsequent code that the emission budget target (```qCO2_EB[t]```) defines the cumulative target in the *last* year of the relevant time horizon.

### 6.3. Growth adjusted variables

Assuming that targets were defined as non-growth adjusted variables, we redefine them here as growth adjusted ones (what the code ultimately assumes):

In [59]:
[db.__setitem__(k, (db(k) * (1+db('g_LR'))**(t0-db(k).index.to_series()))) for k in ('qCO2_SYT','qCO2_LRP','qCO2_EB')];

Next, we introduce trends in the emission intensity. Emission intensites $\mu$ measures preabatement CO2 emissions per unit of output. If emission and output quantities are growth-adjusted, the intensity should not be changed. Instead, we look at emissions trends from 1990 to today (this naturally overestimates the growth rate, as emission taxes has been increasing over this period. Here, we simply attribute roughly half of this to "natural" progress in technology):

Add growth rates to emission intensities:

In [60]:
exogenousShare_gCO2 = 0.5
growthFactor = stdSort(adjMultiIndex.bc((1+db('gCO2')*exogenousShare_gCO2).droplevel('t'), db('t')))
db['uCO2'] = growthFactor.pow(pd.Series(db('t')-t0, index = db('t'))) * db('uCO2').droplevel('t')

## 7. Abatement

### 7.1. Data for ```AbateSimple``` class

To add abatement, we need to include abatement technologies. Ideally, we would have data on specific abatement technologies and their (1) abatement potential, (2) average abatement costs, and (3) some measure of variance for abatement costs. For all three, we would ideally have projections over time as well. In lieu of this, the following creates a sample with dimensions that are consistent with model formulation:

Create abatement technology toy data. This creates a number of CCS-like technologies with costs decreasing over time at a rate $g$ to some year (here 2050):

In [61]:
n, Ttech = 3, T # number of technologies; the year that technical progress in costs stop.
c0 = np.hstack([4000, np.linspace(2000, 1000, n-1)]) # costs in baseline year
pot0 = np.hstack([.6, np.full(n-1, .3)]) # reduction potentials
g = 0.01 # growth rate in costs
costs = pd.DataFrame((np.power(1-g, range(Ttech-t0+1)).reshape(Ttech-t0+1,1) * c0 ), 
                     index = pd.Index(range(t0, Ttech+1), name = 't'), 
                     columns = 'CCS'+pd.Index(range(n), name = 'tech').astype(str)).stack()
pots = pd.DataFrame(np.tile(pot0, (len(db('t')), 1)), index = db('t'), columns = costs.index.levels[1]).stack()
DACCosts= pd.Series(6000, index = db('t')) # keep costs high for DAC in this run

Assume technology applies in all production sectors with emissions and provide dummies:

In [62]:
db['tech'] = costs.index.levels[1] # 'tech' set
db['dTechTau'] = pyDatabases.cartesianProductIndex([db('dtauCO2'), db('tech')]) # combination of [s,n,tech] relevant for abatement costs
db['dtech'] = db('dTechTau').droplevel('n').unique() # combination of [s,tech].
costsFull = stdSort(adjMultiIndex.applyMult(costs, db('dtech')))
potsFull  = stdSort(adjMultiIndex.applyMult(pots,  db('dtech')))
db['techCost'] = extrapolateUpper(costsFull, T) * (db('M1990') * 1e6)/db._scale
db['techPot'] = extrapolateUpper(potsFull, T)
db['DACCost'] = DACCosts * (db('M1990') * 1e6)/db._scale
db['DACSmooth'] = pd.Series(1, index = db('DACCost').index)
db['qCO2Base'] = 0 # remove DAC potential

### 7.2. Data for ```AbateCapital``` class

Relevant data for the ```AbateCapital``` class is already added above or is simply initialized with some GAMS code later.

## X. Globals

Other parameters/variables used throughout:

In [63]:
db.aom(pd.Series(db('R_LR'), index = db('t')), name = 'Rrate', priority = 'first') # fix interest rate path at long run level
db.aom(pd.Series(db('R_LR')*(1+db('infl_LR')), index = db('t')), name = 'iRate', priority = 'first')

## Export

In [64]:
db.export(repo=d['data'])